In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pandas as pd
plt.style.use('ggplot')
%matplotlib inline

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', validation_size=0, one_hot=True)
test_len = mnist.test.images.shape[0]
validation_len = int(test_len * 0)
num_label=10
train_x = mnist.train.images
test_x = mnist.test.images[validation_len : test_len, :]
validation_x = mnist.test.images[ : validation_len, :]

train_y = mnist.train.labels
test_y = mnist.test.labels[validation_len : test_len]
validation_y = mnist.test.labels[ : validation_len]

print("\nTraining X shape: " + str(train_x.shape))
print("Testing X shape: " + str(test_x.shape))
print("Validation X shape: " + str(validation_x.shape))

print("\nTraining Y shape: " + str(train_y.shape))
print("Testing Y shape: " + str(test_y.shape))
print("Validation Y shape: " + str(validation_y.shape))

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz

Training X shape: (60000, 784)
Testing X shape: (10000, 784)
Validation X shape: (0, 784)

Training Y shape: (60000, 10)
Testing Y shape: (10000, 10)
Validation Y shape: (0, 10)


In [ ]:
M = 10
n_intergal_sample = 30
D=7*7*64
h1=512
D2=10

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf.reset_default_graph()
# variables
X = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None,10])

w_conv1 = tf.get_variable('w_conv1', [5,5, 1,32], initializer = tf.contrib.layers.xavier_initializer())
#b_conv1 = bias_variable('b_conv1', [first_conv[3]])
w_conv2 = tf.get_variable('w_conv2', [5,5,32,64], initializer = tf.contrib.layers.xavier_initializer())
#b_conv2 =bias_variable('b_conv2', [second_conv[3]])

w_fc1 = tf.get_variable('w_fc1', [7*7*64, h1], initializer = tf.contrib.layers.xavier_initializer())
#b_fc1 = tf.get_variable('b_fc1', [1,512])
b_fc1 = tf.get_variable('b_fc1', [1,h1], initializer = tf.contrib.layers.xavier_initializer())
w_fc2 = tf.get_variable('w_fc2', [512, 10], initializer = tf.contrib.layers.xavier_initializer())
b_fc2 = tf.get_variable('b_fc2', [1,10], initializer = tf.contrib.layers.xavier_initializer())



eps1 = tf.random_normal(shape=[n_intergal_sample,D,h1], mean=0.0, stddev=1,dtype=tf.float32)
eps2 = tf.random_normal(shape=[n_intergal_sample,1,h1], mean=0.0, stddev=1,dtype=tf.float32)
eps3 = tf.random_normal(shape=[n_intergal_sample,h1,D2], mean=0.0, stddev=1,dtype=tf.float32)
eps4 = tf.random_normal(shape=[n_intergal_sample,1,D2], mean=0.0, stddev=1,dtype=tf.float32)

eps11 = tf.reshape(eps1,[n_intergal_sample,D*h1])
eps22 = tf.reshape(eps2,[n_intergal_sample,1*h1])
eps33 = tf.reshape(eps3,[n_intergal_sample,h1*D2])
eps44 = tf.reshape(eps4,[n_intergal_sample,1*D2])

input_latent = tf.concat([eps11,eps22,eps33,eps44],1)
hidden1 = tf.layers.dense(input_latent, 15, activation=tf.nn.relu)
#hidden2 = tf.layers.dense(hidden1, 5, activation=tf.nn.relu)
#hidden3 = tf.layers.dense(hidden2, 30, activation=tf.nn.relu)
output_latent = tf.layers.dense(hidden1,(D*h1 + 1*h1+ h1*D2 + 1*D2), activation=None)

w0 = tf.reshape(output_latent[:,0:D*h1],[n_intergal_sample,D,h1])
b0 = tf.reshape(output_latent[:,D*h1:D*h1+1*h1],[n_intergal_sample,1,h1])
w1 = tf.reshape(output_latent[:,D*h1+1*h1:D*h1+1*h1+h1*D2],[n_intergal_sample,h1,D2])
b1 = tf.reshape(output_latent[:,D*h1+1*h1+h1*D2:D*h1+1*h1+h1*D2+1*D2],[n_intergal_sample,1,D2])

# network
b=True
con1 = tf.nn.conv2d(X, w_conv1, strides=[1, 1, 1, 1], padding='SAME')
h_conv1 = tf.nn.relu(tf.layers.batch_normalization(con1, training=b))
h_pool1 = max_pool_2x2(h_conv1)

con2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1, 1, 1, 1], padding='SAME')
h_conv2 = tf.nn.relu(tf.layers.batch_normalization(con2, training=b))
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.layers.flatten(h_pool2)
h_pool2_flat1 = tf.stack([h_pool2_flat]*n_intergal_sample)

h = tf.nn.relu(tf.matmul(h_pool2_flat1, w0) + b0)
logits = tf.matmul(h, w1) + b1
output0 = tf.nn.softmax(logits)

probs = tf.reduce_sum(output0*Y,2)
print(probs.shape)
prob=tf.reduce_mean(probs,0)
print(prob.shape)
log_prob = tf.log(prob)
print(log_prob.shape)
p = tf.reduce_mean(log_prob)
print(p.shape)

output = tf.reduce_mean(output0,0)
print(output.shape)
#output = tf.nn.softmax(logits)

likelihood = tf.tensordot(output[0],Y[0],1)
#cross_ent = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(output), reduction_indices=[1]))

#cross_ent=tf.nn.softmax_cross_entropy_with_logits(logits = output, labels=Y)

correct_pred = tf.equal(tf.argmax(output,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


# training
a = 0.00001
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(a).minimize(-p)
saver = tf.train.Saver()

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    model_id=40
    save_path = saver.restore(sess, "./baseline2/classification_mnist39/model.ckpt")
    for i in range(60000*100):
        batch = mnist.train.next_batch(M)
        x_batch = np.reshape(batch[0],[M, 28, 28, 1])
        y_batch = batch[1]
        
        if i%100==0:
            b=False
            #accu = sess.run(accuracy,{X: np.reshape(mnist.train.images,[55000,28,28,1]), Y: mnist.train.labels})
            test_error = sess.run(accuracy, {X:np.reshape(mnist.test.images,[10000,28,28,1]), Y:mnist.test.labels})
            like = sess.run(likelihood,{X: x_batch, Y: y_batch})

            print ("time:",i, time.time() - start_time, "train accuracy", "test accuracy", test_error, like)
        if i < 1000:
            a = 0.00001
            a = 0.000001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 10000:
            a = 0.000005
            a = 0.000001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 60000:
            a = 0.000001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        else:
            a = 0.0000003
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        
        
        if i%500==0:
            save_path = saver.save(sess, "./baseline2/classification_mnist%s/model.ckpt" % model_id)
            model_id+=1

(30, ?)
(?,)
(?,)
()
(?, 10)
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./baseline2/classification_mnist39/model.ckpt
time: 0 27.87461519241333 train accuracy test accuracy 0.9846 0.7907916
time: 100 211.06593418121338 train accuracy test accuracy 0.9816 0.99977434
time: 200 395.04737615585327 train accuracy test accuracy 0.9862 0.9992749
time: 300 576.8495581150055 train accuracy test accuracy 0.9893 0.99641556
time: 400 760.693213224411 train accuracy test accuracy 0.9884 0.9982697
time: 500 942.4445991516113 train accuracy test accuracy 0.9876 0.99115413
time: 600 1124.9078052043915 train accuracy test accuracy 0.986 0.9968909
time: 700 1307.867832183838 train accuracy test accuracy 0.9879 0.99993813
time: 800 1488.715737104416 train accuracy test accuracy 0.987 0.9995757
time: 900 1670.9868264198303 train accuracy test accuracy 0.986 0.9976024
time: 1000 1851.173457145691 train accuracy test accur

In [31]:
D*h1 + 1*h1+ h1*D2 + 1*D2

314710